In [79]:
# 한곳만 주소 체계 다르니.. 
# 한개는 지역코드에 주소안에있는 아무문자나 입력해놓자. 
# 한곳 이랑 나머지 세곳 아예 html 구조 다르니.. 크롤링 두가지로해야함. 

# 이번엔 최대한 로버스트 하게 예외 안뜨게 해보자. 
# 뭔가 에러뜰 상황이면 if주고 걍 크롤링 안해버림 

# 적어도 제목, 날짜는 필수. 없으면 다음영화 넘어감. 

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib import parse

# libGroup = "ydpllc"
libGroup = "dllib"


if libGroup == "ydpllc":
    url = "http://" + libGroup + ".sen.go.kr/ydpllc/board/index.do?menu_idx=16&manage_idx=947"
    request = requests.get(url)
    response = request.text
    soup = BeautifulSoup(response, 'lxml')
    
    movie_tables = soup.select('div.search-results > div.row') # 영화 없으면 아예 아래 포문 안들어가게 하겠슴.
    
    for movie_table in movie_tables:

        title = ""  # 제목
        when = ""  # 일시
        when_date_year = 0
        when_date_month = 0
        when_date_day = 0
        when_time_hour = 0
        when_time_minuite = 0
        runtime = 0  # 런타임
        place = ""  # 장소
        
        
        # 제목 따로 a테그라 먼저 찾겠슴 
        title_pre = movie_table.select_one('div.box > div.item > div.bif > a.name')
        if title_pre:
            title = title_pre.get_text(strip=True)
            print(f'title: {title}')
        else:
            print('영화제목없음')
            continue# 타이틀 없으면 다음 영화로 넘어감. 
            
            
        # 나머지정보들 묶여서 한번에 크롤링 
        item_pres = movie_table.select('div.box > div.item > div.bif > ul.con2 > li')
        
        for item_pre in item_pres:
            item_name_pre = item_pre.get_text()
            
            if "상영일시" in item_name_pre:
            
                when = item_name_pre
                print(f'when: {when}')

                when_date_year_pre = re.findall('(\d\d\d\d)', when)
                if when_date_year_pre:
                    when_date_year = when_date_year_pre[0]
                when_date_month_pre = re.findall('\d\d\d\d\s*-\s*(\d{1,2})', when)
                if when_date_month_pre:
                    when_date_month = when_date_month_pre[0]
                when_date_day_pre = re.findall('\d\d\d\d\s*-\s*\d{1,2}\s*-\s*(\d{1,2})', when)
                if when_date_day_pre:
                    when_date_day = when_date_day_pre[0]
                
                # 18:00 이런식으로 표현시 
                if re.findall('(\d{1,2})\s*:', when):
                    when_time_hour_pre= re.findall('(\d{1,2})\s*:', when)
                    if when_time_hour_pre:
                        when_time_hour = when_time_hour_pre[0]
                    when_time_minuite_pre = re.findall('\d{1,2}\s*:\s*(\d{1,2})\s*', when)
                    if when_time_minuite_pre:
                        when_time_minuite = when_time_minuite_pre[0]
                # 18시 30분 이런식으로 표현시 
                elif re.findall('\d+\s*시\s*(\d+)\s*분', when):
                    when_time_hour_pre = re.findall('(\d+)\s*시\s*\d+\s*분', when)
                    if when_time_hour_pre:
                        when_time_hour = when_time_hour_pre[0]
                    when_time_minuite_pre = re.findall('\d+\s*시\s*(\d+)\s*분', when)
                    if when_time_minuite_pre:
                        when_time_minuite = when_time_minuite_pre[0]

                # 18시 이런식으로 표현될땐 ~
                elif re.findall('(\d+)\s*시', when):
                    when_time_hour_pre = re.findall('(\d+)\s*시', when)
                    if when_time_hour_pre:
                        when_time_hour = when_time_hour_pre[0]

                print(f'when_date_year: {when_date_year}')
                print(f'when_date_month: {when_date_month}')
                print(f'when_date_day: {when_date_day}')
                print(f'when_time_hour: {when_time_hour}')
                print(f'when_time_minuite: {when_time_minuite}')
            
        
            elif "상영장소" in item_name_pre:
                place_pre = item_name_pre.split(':')
                if place_pre:
                    place = place_pre[1]
                    print(f'place: {place}')

            elif "상영시간" in item_name_pre:
                runtime_pre  =re.findall('(\d+)', item_name_pre)
                if runtime_pre:
                    runtime = runtime_pre[0]
                    print(f'runtime: {runtime}')
        
        print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
        
        
        
        
# http://www.dllib.or.kr/program/movieList.do
    
    
    
else:
    url = "http://www." + libGroup + ".or.kr/program/movieList.do"
    request = requests.get(url)
    response = request.text
    soup = BeautifulSoup(response, 'lxml')
    
    movie_tables = soup.select('div.contents > form > table')
        
    for movie_table in movie_tables:
        title = ""  # 제목
        when = ""  # 일시
        when_date_year = 0
        when_date_month = 0
        when_date_day = 0
        when_time_hour = 0
        when_time_minuite = 0
        runtime = 0  # 런타임
        place = ""  # 장소
        
        item_pres = movie_table.select('th')
#         findNext('th').contents[0]

        for item_pre in item_pres:
            
            if item_pre.get_text() == "제목":
                title_pre = item_pre.find_next_sibling()
                if title_pre:
                    title = title_pre.get_text(strip=True)
                    
                    if ']' in title:                         # [우리말 자막]레고 배트맨 : 더 무비 이런경우 앞의 괄호부분없애줌. 
                        title_before = title.split(']')
                        title = title_before[1]  
                    elif '무료영화상영' and '"' in title:   # 무료영화상영 "아기배달부 스토크"  이경우 안쪽만 빼주자.  
                        title_before_pre = re.findall('"(.*)"',title)
                        if title_before_pre:
                            title = title_before_pre[0]
                    print(f'title: {title}')
                else:
                    break # 제목 정보 없으면 다음영화 넘어감 
                
            elif item_pre.get_text() == "상영일시":
                when_pre = item_pre.find_next_sibling()
                if when_pre:
                    when = when_pre.get_text(strip=True)
                    print(f'when: {when}')
                else:
                    break # 날짜 정보 없으면 다음영화 넘어감. 

                when_date_year_pre = re.findall('(\d\d\d\d)', when)
                if when_date_year_pre:
                    when_date_year = when_date_year_pre[0]
                when_date_month_pre = re.findall('\d\d\d\d\s*.\s*(\d{1,2})', when)
                if when_date_month_pre:
                    when_date_month = when_date_month_pre[0]
                when_date_day_pre = re.findall('\d\d\d\d\s*.\s*\d{1,2}\s*.\s*(\d{1,2})', when)
                if when_date_day_pre:
                    when_date_day = when_date_day_pre[0]
                
                # 18:00 이런식으로 표현시 
                if re.findall('(\d{1,2})\s*:', when):
                    when_time_hour_pre= re.findall('(\d{1,2})\s*:', when)
                    if when_time_hour_pre:
                        when_time_hour = when_time_hour_pre[0]
                    when_time_minuite_pre = re.findall('\d{1,2}\s*:\s*(\d{1,2})\s*', when)
                    if when_time_minuite_pre:
                        when_time_minuite = when_time_minuite_pre[0]
                # 18시 30분 이런식으로 표현시 
                elif re.findall('\d+\s*시\s*(\d+)\s*분', when):
                    when_time_hour_pre = re.findall('(\d+)\s*시\s*\d+\s*분', when)
                    if when_time_hour_pre:
                        when_time_hour = when_time_hour_pre[0]
                    when_time_minuite_pre = re.findall('\d+\s*시\s*(\d+)\s*분', when)
                    if when_time_minuite_pre:
                        when_time_minuite = when_time_minuite_pre[0]

                # 18시 이런식으로 표현될땐 ~
                elif re.findall('(\d+)\s*시', when):
                    when_time_hour_pre = re.findall('(\d+)\s*시', when)
                    if when_time_hour_pre:
                        when_time_hour = when_time_hour_pre[0]
                        
                print(f'when_date_year: {when_date_year}')
                print(f'when_date_month: {when_date_month}')
                print(f'when_date_day: {when_date_day}')
                print(f'when_time_hour: {when_time_hour}')
                print(f'when_time_minuite: {when_time_minuite}')
                
            elif item_pre.get_text() == "상영장소":
                place_pre= item_pre.find_next_sibling()
                if place_pre:
                    place = place_pre.get_text(strip=True)
                    print(f'place: {place}')
                
            elif item_pre.get_text() == "상영정보":
                if item_pre.find_next_sibling():
                    info_tuple = item_pre.find_next_sibling().get_text(strip=True) # 98분 / 전체이용가
                    if re.findall('(\d+)', info_tuple):
                        runtime = re.findall('(\d+)', info_tuple)[0]
                        print(f'runtime: {runtime}')
        
        print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    

# 송파구 코드도 추후에 참고하도록.

# libGroup = "SPJ"

# params = {
#         'LIBCODE': libGroup,
#     }
# url = "http://www.splib.or.kr/movie.do?" + parse.urlencode(params) 

# request = requests.get(url)
# response = request.text
# soup = BeautifulSoup(response, 'lxml')

# movie_tables = soup.select('div.contents > table')

# for movie_table in movie_tables:
#     title = ""  # 제목
#     when = ""  # 일시
#     when_date_year = 0
#     when_date_month = 0
#     when_date_day = 0
#     when_time_hour = 0
#     when_time_minuite = 0
#     runtime = 0  # 런타임
#     place = ""  # 장소
    
#     items_pre = movie_table.select('tr')
    
#     for item_pre in items_pre:
#         item_name = item_pre.select_one('th').get_text(strip=True)
        
#         if item_name == "제목":
#             title = item_pre.select_one('td.tb').get_text(strip=True)
#             print(f'title: {title}')
            
#         elif item_name == "상영일시":
#             when = item_pre.select_one('td').get_text(strip=True)
#             print(f'when: {when}')# 2019년 04월 07일 (일요일) 14시 00분

#             when_date_year = re.findall('(\d\d\d\d)', when)[0]
#             when_date_month = re.findall('(\d{1,2})\s*월', when)[0]
#             when_date_day = re.findall('(\d{1,2})\s*일', when)[0]
            
#             # 18:00 이런식으로 표현시 
#             if re.findall('(\d{1,2}):', when):
#                 when_time_hour = re.findall('(\d{1,2}):', when)[0]
#                 when_time_minuite = re.findall('\d{1,2}:(\d{1,2})', when)[0]
#             # 18시 30분 이런식으로 표현시 
#             elif re.findall('\d+\s*시\s*(\d+)\s*분', when):
#                 when_time_hour = re.findall('(\d+)\s*시\s*\d+\s*분', when)[0]
#                 when_time_minuite = re.findall('\d+\s*시\s*(\d+)\s*분', when)[0]

#             # 18시 이런식으로 표현될땐 ~
#             elif re.findall('(\d+)\s*시', when):
#                 when_time_hour = re.findall('(\d+)시', when)[0]
    
#             print(f'when_date_year: {when_date_year}')
#             print(f'when_date_month: {when_date_month}')
#             print(f'when_date_day: {when_date_day}')
#             print(f'when_time_hour: {when_time_hour}')
#             print(f'when_time_minuite: {when_time_minuite}')
            
            
#         elif item_name == "상영장소":
#             place = item_pre.select_one('td').get_text(strip=True)
#             print(f'place: {place}')
            
#         elif item_name == "상영정보":
#             runtime = item_pre.select_one('td').get_text(strip=True)
            
#             if re.findall('(\d+)', runtime):
#                 runtime = re.findall('(\d+)', runtime)[0]

#             print(f'runtime: {runtime}')
    
    
#     print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')

# title: 사운드 오브 뮤직
# when: 2019년 04월 21일
#                           (일요일)

#                           15시 00분
# when_date_year: 2019
# when_date_month: 04
# when_date_day: 21
# when_time_hour: 15
# when_time_minuite: 00
# place: 소나무언덕 잠실본동 작은도서관 2층 다목적실
# runtime: 172

title: 부니베어: 나무 도둑의 습격
when: 2019.06.08 (토) 16:00
when_date_year: 2019
when_date_month: 06
when_date_day: 08
when_time_hour: 16
when_time_minuite: 00
place: 2층 문화강좌실
runtime: 57
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 아기배달부 스토크
when: 2019.06.22 (토) 16:00
when_date_year: 2019
when_date_month: 06
when_date_day: 22
when_time_hour: 16
when_time_minuite: 00
place: 2층 문화강좌실
runtime: 87
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


In [25]:
for i in range(10):
    print(i)
    for j in range(10):
        if j == 4:
            break

0
1
2
3
4
5
6
7
8
9


In [32]:
if 0:
    print(2)